In [1]:
import sys
import os

if sys.version_info[0] < 3:
  print('[ERROR] You need to run this with Python 3.')
  raise AssertionError

In [2]:
import numpy as np

from emtf_algos import *
from emtf_logger import get_logger
from emtf_colormap import get_colormap

In [3]:
# Set random seed
np.random.seed(2027)

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers as k_layers
from tensorflow.keras import backend as k_backend
import matplotlib as mpl
import matplotlib.pyplot as plt

# Set random seed
tf.random.set_seed(2027)

#import numba
#from numba import njit, vectorize
#import dask
#import dask.array as da

logger = get_logger()
logger.info('Using cmssw      : {0}'.format(os.environ['CMSSW_VERSION'] if 'CMSSW_VERSION' in os.environ else 'n/a'))
logger.info('Using python     : {0}'.format(sys.version.replace('\n', '')))
logger.info('Using numpy      : {0}'.format(np.__version__))
logger.info('Using tensorflow : {0}'.format(tf.__version__))
logger.info('Using keras      : {0}'.format(keras.__version__))
logger.info('.. list devices  : {0}'.format(tf.config.list_physical_devices()))
logger.info('Using matplotlib : {0}'.format(mpl.__version__))
#logger.info('Using numba      : {0}'.format(numba.__version__))
#logger.info('Using dask       : {0}'.format(dask.__version__))

assert k_backend.backend() == 'tensorflow'
assert k_backend.image_data_format() == 'channels_last'

%matplotlib inline

[INFO    ] Using cmssw      : CMSSW_10_6_3
[INFO    ] Using python     : 3.6.10 |Anaconda, Inc.| (default, May  8 2020, 02:54:21) [GCC 7.3.0]
[INFO    ] Using numpy      : 1.19.1
[INFO    ] Using tensorflow : 2.2.0
[INFO    ] Using keras      : 2.3.0-tf
[INFO    ] .. list devices  : [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]
[INFO    ] Using matplotlib : 3.2.2


In [4]:
# Settings

# zone: (0,1,2) -> eta=(1.98..2.5, 1.55..1.98, 1.2..1.55)
zone = 0
#zone = 1
#zone = 2

# timezone: (0,1,2) -> BX=(-1,0,+1)
timezone = 1

maxevents = 10
#maxevents = -1

# Input files
patterns_fname = 'patterns_zone%i.npz' % zone
zone_images_fname = 'zone_images_zone%i.h5' % zone

# Styling
plt.style.use('tdrstyle.mplstyle')
cm = get_colormap()

logger.info('Processing zone {0} timezone {1}'.format(zone, timezone))
logger.info('.. maxevents        : {0}'.format(maxevents))

[INFO    ] Processing zone 0 timezone 1
[INFO    ] .. maxevents        : 10


### Load data

In [5]:
def load_patterns():
  patterns = []
  boxes_act = []
  hitmap_quality_ranks = []
  for i in range(num_emtf_zones):
    fname = patterns_fname.replace('zone%i' % zone, 'zone%i' % i)  # modify filename
    logger.info('Loading from {0}'.format(fname))
    with np.load(fname) as loaded:
      patterns.append(loaded['patterns'])
      boxes_act.append(loaded['boxes_act'])
      hitmap_quality_ranks.append(loaded['hitmap_quality_ranks'])
  patterns = np.asarray(patterns)
  boxes_act = np.asarray(boxes_act)
  hitmap_quality_ranks = np.asarray(hitmap_quality_ranks)
  logger.info('patterns: {0} boxes_act: {1} hitmap_quality_ranks: {2}'.format(patterns.shape, boxes_act.shape, hitmap_quality_ranks.shape))
  return patterns, boxes_act, hitmap_quality_ranks

import h5py
loaded_h5 = None  # hdf5 file handle

def load_zone_sparse_images(fname):
  global loaded_h5
  if loaded_h5 is None:
    logger.info('Loading from {0}'.format(fname))
    loaded_h5 = h5py.File(fname, 'r')
  zone_box_anchors = loaded_h5['zone_box_anchors']
  zone_sparse_images = SparseTensorValue(indices=loaded_h5['zone_sparse_images_indices'],
                                         values=loaded_h5['zone_sparse_images_values'],
                                         dense_shape=loaded_h5['zone_sparse_images_dense_shape'])
  logger.info('zone_box_anchors: {0} zone_sparse_images: {1}'.format(zone_box_anchors.shape, zone_sparse_images.dense_shape))
  return zone_box_anchors, zone_sparse_images

def load_zone_hits(fname):
  global loaded_h5
  if loaded_h5 is None:
    logger.info('Loading from {0}'.format(fname))
    loaded_h5 = h5py.File(fname, 'r')
  zone_part = loaded_h5['zone_part']
  zone_hits = RaggedTensorValue(values=loaded_h5['zone_hits_values'],
                                row_splits=loaded_h5['zone_hits_row_splits'])
  zone_simhits = RaggedTensorValue(values=loaded_h5['zone_simhits_values'],
                                   row_splits=loaded_h5['zone_simhits_row_splits'])
  logger.info('zone_part: {0} zone_hits: {1} zone_simhits: {2}'.format(zone_part.shape, zone_hits.shape, zone_simhits.shape))
  return zone_part, zone_hits, zone_simhits

In [6]:
def sparse_to_dense_quick(sparse, maxevents):
  dense_shape = (maxevents,) + sparse.dense_shape[1:]
  dense = np.zeros(dense_shape, dtype=sparse.dtype)
  for i in range(len(sparse.indices)):
    if sparse.indices[i, 0] >= maxevents:
      break
    tup = tuple(sparse.indices[i])
    dense[tup] = sparse.values[i]
  return dense

In [7]:
patterns, boxes_act, hitmap_quality_ranks = load_patterns()

# Create boxes_act_reshaped
boxes_act_reshaped = []
for i in range(num_emtf_zones):
  b = boxes_act[i, 3, [3, 2, 4, 1, 5, 0, 6]]  # order by straightness, only prompt patterns
  b = np.transpose(b, [3, 2, 1, 0])  # kernel shape is HWCD
  boxes_act_reshaped.append(b)
boxes_act_reshaped = np.asarray(boxes_act_reshaped)
logger.info('boxes_act_reshaped: {0}'.format(boxes_act_reshaped.shape))

# Create boxes_qual_reshaped
boxes_qual_reshaped = hitmap_quality_ranks // 4  # from 8-bit to 6-bit
assert boxes_qual_reshaped.max() == 63
logger.info('boxes_qual_reshaped: {0}'.format(boxes_qual_reshaped.shape))

[INFO    ] Loading from patterns_zone0.npz
[INFO    ] Loading from patterns_zone1.npz
[INFO    ] Loading from patterns_zone2.npz
[INFO    ] patterns: (3, 7, 7, 8, 3) boxes_act: (3, 7, 7, 8, 111, 1) hitmap_quality_ranks: (3, 256)
[INFO    ] boxes_act_reshaped: (3, 1, 111, 8, 7)
[INFO    ] boxes_qual_reshaped: (3, 256)


In [8]:
zone_box_anchors, zone_sparse_images = load_zone_sparse_images(zone_images_fname)

zone_images_test = sparse_to_dense_quick(zone_sparse_images, maxevents)
zone_box_anchors_test = zone_box_anchors[:maxevents]
logger.info('zone_box_anchors_test: {0} zone_images_test: {1}'.format(zone_box_anchors_test.shape, zone_images_test.shape))

[INFO    ] Loading from zone_images_zone0.h5
[INFO    ] zone_box_anchors: (652055,) zone_sparse_images: (652055, 8, 288, 1)
[INFO    ] zone_box_anchors_test: (10,) zone_images_test: (10, 8, 288, 1)


In [9]:
zone_part, zone_hits, zone_simhits = load_zone_hits(zone_images_fname)

[INFO    ] zone_part: (652055, 9) zone_hits: (652055, None, 16) zone_simhits: (652055, None, 16)


### Create inputs

In [10]:
image_format = zone_images_test.shape[1:]
(num_rows, num_cols, num_channels) = image_format

num_patterns = 7
num_out_tracks = 4
num_out_variables = 36
num_embedding_input_dim = (2 ** num_rows)

hits_metadata = ['emtf_layer', 'ri_layer', 'zones', 'timezones',
                 'emtf_chamber', 'emtf_segment', 'detlayer', 'bx',
                 'emtf_phi', 'emtf_bend', 'emtf_theta', 'emtf_theta_alt',
                 'emtf_qual', 'emtf_time', 'fr', 'rsvd']
hits_metadata = dict(zip(hits_metadata, range(len(hits_metadata))))
#print(hits_metadata)

ind_emtf_chamber = hits_metadata['emtf_chamber']
ind_emtf_segment = hits_metadata['emtf_segment']

ind_emtf_phi = hits_metadata['emtf_phi']
ind_emtf_bend = hits_metadata['emtf_bend']
ind_emtf_theta = hits_metadata['emtf_theta']
ind_emtf_theta_alt = hits_metadata['emtf_theta_alt']
ind_emtf_qual = hits_metadata['emtf_qual']
ind_emtf_time = hits_metadata['emtf_time']
ind_zones = hits_metadata['zones']
ind_timezones = hits_metadata['timezones']
ind_bx = hits_metadata['bx']
ind_valid = hits_metadata['rsvd']  # CUIDADO: use 'rsvd' for the moment

In [11]:
def create_inputs():
  inputs = []
  sparse_inputs = []
  for ievt in range(zone_hits.shape[0]):
    if maxevents != -1 and ievt == maxevents:
      break

    dense_shape = np.array([num_emtf_chambers, num_emtf_segments, num_emtf_variables], dtype=np.int32)
    zone_hits_columns = [ind_emtf_chamber, ind_emtf_segment]
    indices = zone_hits[ievt][:, zone_hits_columns]
    zone_hits_columns = [ind_emtf_phi, ind_emtf_bend, ind_emtf_theta, ind_emtf_theta_alt, ind_emtf_qual, ind_emtf_time,
                         ind_zones, ind_timezones, ind_bx, ind_valid]
    values = zone_hits[ievt][:, zone_hits_columns]
    values[:, -1] = 1  # CUIDADO: set 'valid' to 1
    #print(dense_shape)
    #print(indices.shape, indices)
    #print(values.shape, values)

    # Apply truncation
    valid = indices[:, 1] < num_emtf_segments
    indices = indices[valid]
    values = values[valid]

    # Mimic sparse_to_dense()
    ndims = indices.shape[1]
    tup = tuple(indices[: ,i] for i in range(ndims))
    dense = np.zeros(dense_shape, dtype=values.dtype)
    dense[tup] = values
    inputs.append(dense)
    sparse_inputs.append(np.concatenate((indices, values), axis=-1))
  return np.asarray(inputs), sparse_inputs

In [12]:
inputs, sparse_inputs = create_inputs()

logger.info('inputs: {0} sparse_inputs: ({1}, None, {2})'.format(inputs.shape, len(sparse_inputs), sparse_inputs[0].shape[-1]))

[INFO    ] inputs: (10, 115, 8, 10) sparse_inputs: (10, None, 12)


In [13]:
# Debug
print(np.array2string(sparse_inputs[0], separator=', ', threshold=1000))
print(np.array2string(sparse_inputs[2], separator=', ', threshold=1000))

[[   2,    0, 2548,    5,   18,   17,   -6,    0,    4,    3,    0,    1],
 [   2,    1, 2548,    5,   17,   18,   -6,    0,    4,    3,    0,    1],
 [  19,    0, 2684,    2,   16,   16,    6,    0,    4,    3,    0,    1],
 [  28,    0, 2819,   15,   17,   16,   -5,    0,    4,    3,    0,    1],
 [  28,    1, 2728,    0,   16,   17,   -5,    0,    4,    3,    0,    1],
 [  37,    0, 2736,    0,   16,   16,   -5,    0,    4,    3,    0,    1],
 [  55,    0, 2505,    0,   18,   18,    0,    0,    4,    2,    0,    1],
 [  73,    0, 2675,    0,   19,   19,    0,    0,    4,    2,    0,    1],
 [  82,    0, 2888,    0,   17,   17,    0,    1,    4,    2,    0,    1],
 [  82,    1, 2714,    0,   17,   17,    0,    1,    4,    2,    0,    1],
 [  91,    0, 2737,    0,   17,   17,    0,    0,    4,    2,    0,    1],
 [ 109,    0, 2479,   15,   17,   17,    6,    0,    4,    2,    0,    1]]
[[  29,    0, 4503,    1,   11,   11,    5,    0,    4,    6,   -1,    1],
 [  29,    1, 4503,    1,

### Create model

In [14]:
def build_zone_images(x, zone=zone, image_format=image_format):
  # Utility functions & LUTs
  inverse_fn = lambda F, y: [[i for (i, y_i) in enumerate(F) if y_i == y_j] for y_j in y]
  to_array = lambda x: np.asarray([np.asarray(x_i) for x_i in x])
  to_list = lambda x: [x_i.tolist() for x_i in x]
  flatten = lambda x: np.asarray([x_i_i for x_i in x for x_i_i in x_i])

  def to_array(x):  # improved version
    is_ragged = len(set([len(x_i) for x_i in x])) > 1
    if is_ragged:
      return np.asarray([np.asarray(x_i) for x_i in x], dtype=np.object)
    else:
      return np.asarray([x_i for x_i in x])

  num_emtf_ri_layers = 19
  ri_layer_to_chamber_lut = to_array(inverse_fn(chamber_to_ri_layer_lut, range(num_emtf_ri_layers)))
  #ri_layer_to_chamber_lut_flat = flatten(ri_layer_to_chamber_lut)

  num_emtf_zo_layers = 8
  ri_layer_to_zo_layer_lut = find_emtf_zo_layer_lut()[:, zone]
  zo_layer_to_ri_layer_lut = to_array(inverse_fn(ri_layer_to_zo_layer_lut, range(num_emtf_zo_layers)))

  zo_layer_to_chamber_lut = to_array([
      [c for ri_layer in ri_layers for c in ri_layer_to_chamber_lut[ri_layer]] \
      for ri_layers in zo_layer_to_ri_layer_lut
  ])

  def get_boolean_mask(zo_layer):
    indices = zo_layer_to_chamber_lut[zo_layer]
    boolean_mask = np.zeros(num_emtf_chambers, dtype=np.bool)
    boolean_mask[indices] = 1
    return boolean_mask

  # Prepare zone images
  zone_images = np.zeros((x.shape[0],) + image_format, dtype=np.bool)

  # Loop over events
  for ievt in range(zone_images.shape[0]):
    x_emtf_phi = x[ievt][..., 0]  # CUIDADO: hardcoded index
    x_zones = x[ievt][..., 6]     # CUIDADO: hardcoded index
    x_timezones = x[ievt][..., 7] # CUIDADO: hardcoded index
    x_valid = x[ievt][..., 9]     # CUIDADO: hardcoded index

    valid = (x_valid == 1) & \
            (x_zones & (1<<(2-zone))).astype(np.bool) & \
            (x_timezones & (1<<(2-timezone))).astype(np.bool)
    zo_phi = find_emtf_zo_phi(x_emtf_phi)

    # Loop over rows
    rows = []
    cols = []
    channels = []
    for zo_layer in range(zone_images.shape[1]):
      boolean_mask = get_boolean_mask(zo_layer)
      _valid = valid[boolean_mask]
      _zo_phi = zo_phi[boolean_mask][_valid]
      rows.extend((_zo_phi * 0) + zo_layer)
      cols.extend(_zo_phi)
      channels.extend((_zo_phi * 0))

    # Fill zone image
    zone_images[ievt][(rows, cols, channels)] = 1
  return zone_images

In [15]:
# Creating custom layers
# See: https://www.tensorflow.org/tutorials/customization/custom_layers

class Zoning(k_layers.Layer):
  def __init__(self, zone, image_format=image_format, **kwargs):
    super(Zoning, self).__init__(**kwargs)
    self.zone = zone
    self.image_format = image_format

    # Set up to call build_zone_images()
    import functools
    kwargs = dict(zone=self.zone, image_format=self.image_format)
    _build_zone_images = functools.partial(build_zone_images, **kwargs)
    #py_func = lambda x: tf.py_function(_build_zone_images, [x], tf.bool)
    py_func = lambda x: tf.numpy_function(_build_zone_images, [x], tf.bool)
    self.py_func = k_layers.Lambda(py_func)

  def call(self, inputs):
    x = inputs
    x = tf.cast(x, dtype=tf.int32)
    x = self.py_func(x)
    x = tf.cast(x, dtype=inputs.dtype)
    output_shape = (None,) + self.image_format
    x.set_shape(output_shape)
    return x

class Pooling(k_layers.Layer):
  def __init__(self, zone, image_format=image_format, num_patterns=num_patterns, **kwargs):
    super(Pooling, self).__init__(**kwargs)
    self.zone = zone
    self.image_format = image_format
    self.num_patterns = num_patterns

    # SeparableConv2D but only the depthwise conv (i.e. without the pointwise conv)
    # See: https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv2D
    # See: https://www.tensorflow.org/api_docs/python/tf/keras/layers/SeparableConv2D
    from k_layers_separable_conv2d import SeparableConv2D as MySeparableConv2D
    w_init = tf.keras.initializers.Constant(boxes_act_reshaped[self.zone])
    conv2d_kwargs = dict(filters=1, kernel_size=(boxes_act_reshaped.shape[1], boxes_act_reshaped.shape[2]), depth_multiplier=self.num_patterns,
                         strides=(1, 1), padding='same', activation=None, use_bias=False,
                         depthwise_initializer=w_init, pointwise_initializer='ones', trainable=False)
    self.conv2d = MySeparableConv2D(**conv2d_kwargs)

    # Embedding
    # See: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
    w_init = tf.keras.initializers.Constant(boxes_qual_reshaped[self.zone])
    embedding_kwargs = dict(input_dim=num_embedding_input_dim, output_dim=1, input_length=1,
                            embeddings_initializer=w_init, trainable=False)
    self.embedding = k_layers.Embedding(**embedding_kwargs)

    # Dot product coeffs for packing the last axis
    self.po2_coeffs = (2 ** np.arange(self.image_format[0]))  # [1,2,4,8,16,32,64,128]
    self.po2_coeffs = self.po2_coeffs.astype(np.int32)

  def call(self, inputs):
    # Conv
    x = inputs  # NHWC, which is (None, 8, 288, 1)
    x = tf.transpose(x, perm=(0, 3, 2, 1))  # NHWC -> NCWH
    x = self.conv2d(x)  # NCWH -> NCWH', H' is dim of size H * D, D is depth_multipler
    x = tf.reshape(x, [-1, self.image_format[2], self.image_format[1], self.image_format[0], self.num_patterns])  # NCWH' -> NCWHD
    x = tf.transpose(x, perm=(0, 1, 2, 4, 3))  # NCWHD -> NCWDH
    x = tf.reduce_sum(x, axis=1)  # NCWDH -> NWDH, C is dim of size 1 and has been dropped

    # Pack 8 bits into a single number
    x = tf.clip_by_value(x, 0, 1)
    x = tf.cast(x, dtype=tf.int32)
    x = tf.reduce_sum(x * self.po2_coeffs, axis=-1)  # NWDH -> NWD, H has been packed into a single number and dropped
    x = tf.cast(x, dtype=inputs.dtype)

    # Embedding
    x = self.embedding(x)  # NWD -> NWDE, E is embedding output dim
    x = tf.reduce_sum(x, axis=-1)  # NWDE -> NWD, E is dim of size 1 and has been dropped

    # Gather max element
    idx = tf.argmax(x, axis=-1, output_type=tf.int32)  # NWD -> NW
    x = tf.gather(x, idx, axis=-1, batch_dims=2)  # NWD -> NW
    return (x, idx)

class Suppression(k_layers.Layer):
  def __init__(self, zone, image_format=image_format, **kwargs):
    super(Suppression, self).__init__(**kwargs)
    self.zone = zone
    self.image_format = image_format

  def call(self, inputs):
    x, idx = inputs

    # Non-max suppression
    x_padded = tf.pad(x, paddings=((0, 0), (1, 1)))  # ((pad_t, pad_b), (pad_l, pad_r))
    mask = (x > x_padded[:, :-2]) & (x >= x_padded[:, 2:])  # x > x_left && x >= x_right
    mask = tf.cast(mask, dtype=x.dtype)
    x = x * mask
    return (x, idx)

class ZoneSorting(k_layers.Layer):
  def __init__(self, zone, num_out_tracks=num_out_tracks, **kwargs):
    super(ZoneSorting, self).__init__(**kwargs)
    self.zone = zone
    self.num_out_tracks = num_out_tracks

  def call(self, inputs):
    x, idx = inputs

    # Sort (descending)
    tmp_idx = tf.argsort(x, axis=-1, direction='DESCENDING', stable=True)
    tmp_idx.set_shape(x.shape)
    tmp_idx = tf.transpose(tf.transpose(tmp_idx)[:self.num_out_tracks])  # truncate

    # Gather max elements
    x = tf.gather(x, tmp_idx, axis=-1, batch_dims=1)  # NW -> NW', W' is dim of size num_out_tracks
    idx = tf.gather(idx, tmp_idx, axis=-1, batch_dims=1)  # NW -> NW', W' is dim of size num_out_tracks
    return (x, idx)

class ZoneMerging(k_layers.Layer):
  def __init__(self, num_out_tracks=num_out_tracks, **kwargs):
    super(ZoneMerging, self).__init__(**kwargs)
    self.num_out_tracks = num_out_tracks

  def call(self, inputs):
    x, idx = inputs

    # Sort (descending)
    tmp_idx = tf.argsort(x, axis=-1, direction='DESCENDING', stable=True)
    tmp_idx.set_shape(x.shape)
    tmp_idx = tf.transpose(tf.transpose(tmp_idx)[:self.num_out_tracks])  # truncate

    # Gather max elements
    x = tf.gather(x, tmp_idx, axis=-1, batch_dims=1)  # NW -> NW', W' is dim of size num_out_tracks
    idx_p = tf.gather(idx, tmp_idx, axis=-1, batch_dims=1)  # NW -> NW', W' is dim of size num_out_tracks
    idx_z = tmp_idx // self.num_out_tracks
    return (x, idx_p, idx_z)

class TrackBuilding(k_layers.Layer):
  def __init__(self, num_out_variables=num_out_variables, **kwargs):
    super(TrackBuilding, self).__init__(**kwargs)
    self.num_out_variables=num_out_variables

  def call(self, inputs):
    x, idx_p, idx_z = inputs

    # Dummy
    dummy = tf.zeros_like(x, dtype=x.dtype)
    dummy = tf.expand_dims(dummy, -1)
    dummy = tf.tile(dummy, multiples=(1, 1, self.num_out_variables))

    return dummy

In [16]:
def create_model():
  # Input
  inputs = keras.Input(shape=(num_emtf_chambers, num_emtf_segments, num_emtf_variables), name='inputs')
  x = inputs

  # Loop over zones
  x_list = []

  for i in range(num_emtf_zones):
    # Make zone images
    x_i = Zoning(zone=i, name='zoning_{0}'.format(i))(x)

    # Pattern recognition
    x_i = Pooling(zone=i, name='pooling_{0}'.format(i))(x_i)
    x_i = Suppression(zone=i, name='suppression_{0}'.format(i))(x_i)

    # Zone sorter
    x_i = ZoneSorting(zone=i, name='zonesorting_{0}'.format(i))(x_i)

    # Add x_i to x_list
    x_list.append(x_i)

  # Merge zone outputs
  i = 0
  x = (k_layers.Concatenate(axis=-1)([x[0] for x in x_list]),
       k_layers.Concatenate(axis=-1)([x[1] for x in x_list]))
  x = ZoneMerging(name='zonemerging_{0}'.format(i))(x)

  # Track builder
  x = TrackBuilding(name='trkbuilding_{0}'.format(i))(x)

  # Output
  outputs = x

  # Model
  model = keras.Model(inputs=inputs, outputs=outputs, name='awesome_model')

  # Summary
  model.summary()
  return model

In [17]:
model = create_model()

print('trainable weights:', len(model.trainable_weights))
print('all weights:', len(model.weights))

Model: "awesome_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 115, 8, 10)] 0                                            
__________________________________________________________________________________________________
zoning_0 (Zoning)               (None, 8, 288, 1)    0           inputs[0][0]                     
__________________________________________________________________________________________________
zoning_1 (Zoning)               (None, 8, 288, 1)    0           inputs[0][0]                     
__________________________________________________________________________________________________
zoning_2 (Zoning)               (None, 8, 288, 1)    0           inputs[0][0]                     
______________________________________________________________________________________

### Evaluate model

In [18]:
outputs = model(inputs)

logger.info('outputs: {0} type: {1}'.format(outputs.shape, type(outputs)))

[INFO    ] outputs: (10, 4, 36) type: <class 'tensorflow.python.framework.ops.EagerTensor'>


In [19]:
# Debug
model_zoning_0 = keras.Model(inputs=model.input,
                             outputs=model.get_layer('zoning_0').output)
outputs = model_zoning_0(inputs)
print('outputs: {0} type: {1}'.format(outputs.shape, type(outputs)))

if isinstance(outputs, tf.Tensor):
  x = outputs.numpy()
else:
  x = outputs

with np.printoptions(linewidth=100, threshold=1000):
  print(x[0].nonzero())
  print(x[2].nonzero())

outputs: (10, 8, 288, 1) type: <class 'tensorflow.python.framework.ops.EagerTensor'>
(array([0, 1, 2, 3, 4, 5, 5, 6, 6, 7]), array([127, 129, 132, 140, 140, 143, 149, 142, 153, 144]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
(array([0, 2, 3, 4, 5, 6, 7]), array([264, 263, 257, 256, 254, 253, 253]), array([0, 0, 0, 0, 0, 0, 0]))


In [20]:
# Debug
model_pooling_0 = keras.Model(inputs=model.input,
                              outputs=model.get_layer('pooling_0').output)
outputs = model_pooling_0(inputs)
print('outputs: {0} type: {1}'.format(outputs, type(outputs)))

if isinstance(outputs[0], tf.Tensor):
  x = outputs[0].numpy()
else:
  x = outputs[0]

with np.printoptions(linewidth=100, threshold=1000):
  print(x[0].nonzero())
  print(x[2].nonzero())

outputs: (<tf.Tensor: shape=(10, 288), dtype=float32, numpy=
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 31., 31., 31.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)>, <tf.Tensor: shape=(10, 288), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 5, 5, 5],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>) type: <class 'tuple'>
(array([ 93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 

In [21]:
# Debug
model_suppression_0 = keras.Model(inputs=model.input,
                                  outputs=model.get_layer('suppression_0').output)
outputs = model_suppression_0(inputs)
print('outputs: {0} type: {1}'.format(outputs, type(outputs)))

if isinstance(outputs[0], tf.Tensor):
  x = outputs[0].numpy()
else:
  x = outputs[0]

with np.printoptions(linewidth=100, threshold=1000):
  print(x[0].nonzero(), x[0][x[0].nonzero()])
  print(x[2].nonzero(), x[2][x[2].nonzero()])

outputs: (<tf.Tensor: shape=(10, 288), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(10, 288), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 5, 5, 5],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>) type: <class 'tuple'>
(array([ 93,  97, 106, 123, 129, 133, 139]),) [22. 26. 38. 38. 38. 38. 63.]
(array([230, 237, 244, 248, 251, 255]),) [22. 31. 32. 37. 48. 63.]


In [22]:
# Debug
model_zonesorting_0 = keras.Model(inputs=model.input,
                                  outputs=model.get_layer('zonesorting_0').output)
outputs = model_zonesorting_0(inputs)
print('outputs: {0} type: {1}'.format(outputs, type(outputs)))

if isinstance(outputs[0], tf.Tensor):
  x = outputs[0].numpy()
else:
  x = outputs[0]

with np.printoptions(linewidth=100, threshold=1000):
  print(x[0].nonzero(), x[0][x[0].nonzero()])
  print(x[2].nonzero(), x[2][x[2].nonzero()])

outputs: (<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[63., 38., 38., 38.],
       [61., 31., 31., 31.],
       [63., 48., 37., 32.],
       [63., 55., 39., 38.],
       [62., 58., 39., 38.],
       [63., 55., 29.,  4.],
       [63., 48., 38., 38.],
       [55., 39., 38., 38.],
       [63., 48., 37., 22.],
       [63., 55., 47., 39.]], dtype=float32)>, <tf.Tensor: shape=(10, 4), dtype=int32, numpy=
array([[3, 6, 2, 0],
       [5, 6, 2, 0],
       [2, 6, 6, 6],
       [5, 3, 1, 6],
       [4, 6, 6, 6],
       [6, 6, 6, 5],
       [1, 3, 6, 0],
       [6, 6, 2, 5],
       [6, 6, 6, 6],
       [4, 6, 6, 6]], dtype=int32)>) type: <class 'tuple'>
(array([0, 1, 2, 3]),) [63. 38. 38. 38.]
(array([0, 1, 2, 3]),) [63. 48. 37. 32.]


In [23]:
# Debug
model_zonemerging_0 = keras.Model(inputs=model.input,
                                  outputs=model.get_layer('zonemerging_0').output)
outputs = model_zonemerging_0(inputs)
print('outputs: {0} type: {1}'.format(outputs, type(outputs)))

if isinstance(outputs[0], tf.Tensor):
  x = outputs[0].numpy()
else:
  x = outputs[0]

with np.printoptions(linewidth=100, threshold=1000):
  print(x[0].nonzero(), x[0][x[0].nonzero()])
  print(x[2].nonzero(), x[2][x[2].nonzero()])

outputs: (<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[63., 38., 38., 38.],
       [61., 31., 31., 31.],
       [63., 48., 37., 32.],
       [63., 55., 39., 38.],
       [62., 58., 39., 38.],
       [63., 55., 29.,  4.],
       [63., 48., 38., 38.],
       [55., 39., 38., 38.],
       [63., 48., 37., 22.],
       [63., 55., 47., 39.]], dtype=float32)>, <tf.Tensor: shape=(10, 4), dtype=int32, numpy=
array([[3, 6, 2, 0],
       [5, 6, 2, 0],
       [2, 6, 6, 6],
       [5, 3, 1, 6],
       [4, 6, 6, 6],
       [6, 6, 6, 5],
       [1, 3, 6, 0],
       [6, 6, 2, 5],
       [6, 6, 6, 6],
       [4, 6, 6, 6]], dtype=int32)>, <tf.Tensor: shape=(10, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)>) type: <class 'tuple'>
(array([0, 1, 2, 3]),) [63. 38. 38. 38.]
(array([0, 1, 2, 3]),) [